## fiquei com pena de apagar do outro e salvei aqui

Exemplo teste1:

$$
\text{max } Z = 2x_1 + 10x_2 + 8x_3 + 7x_4 + 10x_5 + 10x_6 + 6x_7
$$

$$
\begin{gather*}
5x_1 + 7x_2 + 8x_3 + 1x_4 + 7x_5 + 5x_6 + 6x_7\leq 20 \\
1x_1 + 6x_2 + 4x_3 + 9x_4 + 10x_5 + 6x_6 + 10x_7 \leq 30 \\
4x_1 + 4x_2 + 4x_3 + 1x_4 + 5x_5 + 5x_6 + 10x_7 \leq 40 \\
3x_1 + 10x_2 + 8x_3 + 1x_4 + 3x_5 + 3x_6 + 8x_7\leq 30 \\
10x_1 + 8x_2 + 9x_3 + 9x_4 + 7x_5 + 6x_6 + 10x_7\leq 20 \\
6x_1 + 6x_2 + 3x_3 + 6x_4 + 3x_5 + 7x_6 + 2x_7\leq 80 \\
7x_1 + 10x_2 + 7x_3 + 8x_4 + 7x_5 + 8x_6 + 7x_7\leq 100 \\
9x_1 + 8x_2 + 1x_3 + 1x_4 + 8x_5 + 10x_6 + 2x_7\leq 90 \\
1x_1 + 5x_2 + 3x_3 + 10x_4 + 2x_5 + 4x_6 + 9x_7\leq 70 \\
9x_1 + 6x_2 + 1x_3 + 4x_4 + 7x_5 + 5x_6 + 10x_7\leq 60 \\
5x_1 + 7x_2 + 4x_3 + 4x_4 + 3x_5 + 4x_6 + 10x_7\leq 40 \\
x_1, x_2, x_3, x_4, x_5, x_6, x_7 \geq 0 \\
\end{gather*}
$$
<div style="text-align: center; color: red;"><s>$x_1, x_2, x_3, x_4, x_5, x_6, x_7 \in \{0, 1\}$</s></div>


s.a. 
$$
\begin{gather*}
5x_1 + 7x_2 + 8x_3 + 1x_4 + 7x_5 + 5x_6 + 6x_7\leq 20 \\
1x_1 + 6x_2 + 4x_3 + 9x_4 + 10x_5 + 6x_6 + 10x_7 \leq 30 \\
4x_1 + 4x_2 + 4x_3 + 1x_4 + 5x_5 + 5x_6 + 10x_7 \leq 40 \\
3x_1 + 10x_2 + 8x_3 + 1x_4 + 3x_5 + 3x_6 + 8x_7\leq 30 \\
10x_1 + 8x_2 + 9x_3 + 9x_4 + 7x_5 + 6x_6 + 10x_7\leq 20 \\
6x_1 + 6x_2 + 3x_3 + 6x_4 + 3x_5 + 7x_6 + 2x_7\leq 80 \\
7x_1 + 10x_2 + 7x_3 + 8x_4 + 7x_5 + 8x_6 + 7x_7\leq 100 \\
9x_1 + 8x_2 + 1x_3 + 1x_4 + 8x_5 + 10x_6 + 2x_7\leq 90 \\
1x_1 + 5x_2 + 3x_3 + 10x_4 + 2x_5 + 4x_6 + 9x_7\leq 70 \\
9x_1 + 6x_2 + 1x_3 + 4x_4 + 7x_5 + 5x_6 + 10x_7\leq 60 \\
5x_1 + 7x_2 + 4x_3 + 4x_4 + 3x_5 + 4x_6 + 10x_7\leq 40 \\
x_1, x_2, x_3, x_4, x_5, x_6, x_7 \geq 0 \\
x_1, x_2, x_3, x_4, x_5, x_6, x_7 \in \{0, 1\}
\end{gather*}
$$

## codigo felly

In [2]:
from mip import Model, MAXIMIZE, xsum
from numpy import asarray, abs
import re

def ler_txt(caminho_arquivo):
    with open(caminho_arquivo) as arquivo:
        variaveis, restricoes = map(int, arquivo.readline().split())
        coef_obj = list(map(int, arquivo.readline().split()))

        coef_restricoes = []
        for i in range(restricoes):
            coef_restricoes.append(list(map(int, arquivo.readline().split())))

    return variaveis, restricoes, coef_obj, coef_restricoes

def podar(modelo, primal):
    """
    Verifica se um nó na árvore de busca pode ser podado com base em diferentes critérios.

    Args:
        modelo: O modelo a ser avaliado.
        primal: O valor primal atual.

    Returns:
        str: Um dos seguintes valores: 'INVIAVEL', 'INTEIRO', 'LIMITANTE' ou 'VIÁVEL'.
    """
    modelo.optimize()
    parametros = {}
    parametros["objetivo"] = modelo.objective_value #tudo que tinha resolver agr tem q passar isso amou
    parametros["variaveis"] = modelo.vars
    
    aux_solver = parametros
    count_int = 0

    if aux_solver["objetivo"] == None:
        return 'INVIAVEL'

    for i in aux_solver["variaveis"]:
         if round(i.x) == i.x:  #verifica se é inteiro (mudei tb)
            count_int += 1

    if count_int == len(aux_solver["variaveis"]):
        return 'INTEIRO'

    if aux_solver["objetivo"] <= primal:
        return 'LIMITANTE'
    
    return 'VIÁVEL'


def ramificar(modelo, valores_solucao):
    
    valores_x = [i.x for i in valores_solucao]
    idx = (abs(asarray(valores_x) - 0.5)).argmin() #tirei achar mais proximo e tem q colocar isso no lugar
    var_ramificacao = valores_solucao[idx]

    modelo_0 = modelo.copy()
    modelo_0 += var_ramificacao == 0

    modelo_1 = modelo.copy()
    modelo_1 += var_ramificacao == 1

    return modelo_0, modelo_1


def branch_and_bound(modelo):
    
    global primal
    global modelo_otimo
    
    primal = 0
    modelo_otimo = None
    
    nos = [modelo]

    while nos != []:
        
        nos[0].optimize()
        parametros = {}
        parametros["objetivo"] = nos[0].objective_value #substitui resolver
        parametros["variaveis"] = nos[0].vars
        
        modelo_solver = parametros
        aux = podar(nos[0], primal)
        if aux == 'INVIAVEL' or aux == 'LIMITANTE':
            nos.pop(0)
            
        elif aux == 'INTEIRO':
            if modelo_solver["objetivo"] >= primal:
                modelo_otimo = nos[0]
                primal = modelo_solver["objetivo"]
            nos.pop(0)
            
        elif aux == 'VIÁVEL':
            ramos = ramificar(nos[0], modelo_solver["variaveis"])
            nos.append(ramos[0])
            nos.append(ramos[1])
            nos.pop(0)

def construir_modelo(variaveis, restricoes, coef_obj, coef_restricoes):
    
    modelo = Model(sense=MAXIMIZE)

    x = [modelo.add_var(var_type="CONTINUOUS", lb=0, ub=1, name="X." + str(i+1)) for i in range(variaveis)]

    modelo.objective = xsum(coef_obj[i] * x[i] for i in range(variaveis))

    for i in range(restricoes):
        modelo += xsum(coef_restricoes[i][j] * x[j] for j in range(variaveis)) <= coef_restricoes[i][-1]
    return  modelo   
            
def executar(arquivo):
    
    variaveis, restricoes, coef_obj, coef_restricoes = ler_txt(arquivo)

    modelo = construir_modelo(variaveis, restricoes, coef_obj, coef_restricoes)

    branch_and_bound(modelo)
        
    modelo_otimo.optimize()
    modelo_resolvido = {}
    modelo_resolvido["objetivo"] = modelo_otimo.objective_value
    modelo_resolvido["variaveis"] = modelo_otimo.vars   
    
    print("Modelo:")
    modelo.write("modelo.lp") # salva modelo em arquivo
    with open("modelo.lp", "r") as f: # lê e exibe conteúdo do arquivo
        print(f.read())
    print("\nResultado:")
    print("Variáveis:")
    for i in modelo_resolvido["variaveis"]:
        print(i.name, ' = ', i.x)
    print("Função objetivo:")
    print('Z = ', modelo_resolvido["objetivo"])

executar("teste1.txt")

Modelo:
\Problem name: 

Minimize
OBJROW: -2 X.1 -10 X.2 -8 X.3 -7 X.4 -10 X.5 -10 X.6 -6 X.7
Subject To
constr(0):  5 X.1 + 7 X.2 + 8 X.3 + X.4 + 7 X.5 + 5 X.6 + 6 X.7 <= 20
constr(1):  X.1 + 6 X.2 + 4 X.3 + 9 X.4 + 10 X.5 + 6 X.6 + 10 X.7 <= 30
constr(2):  4 X.1 + 4 X.2 + 4 X.3 + X.4 + 5 X.5 + 5 X.6 + 10 X.7 <= 40
constr(3):  3 X.1 + 10 X.2 + 8 X.3 + X.4 + 3 X.5 + 3 X.6 + 8 X.7 <= 30
constr(4):  10 X.1 + 8 X.2 + 9 X.3 + 9 X.4 + 7 X.5 + 6 X.6 + 10 X.7 <= 20
constr(5):  6 X.1 + 6 X.2 + 3 X.3 + 6 X.4 + 3 X.5 + 7 X.6 + 2 X.7 <= 80
constr(6):  7 X.1 + 10 X.2 + 7 X.3 + 8 X.4 + 7 X.5 + 8 X.6 + 7 X.7 <= 100
constr(7):  9 X.1 + 8 X.2 + X.3 + X.4 + 8 X.5 + 10 X.6 + 2 X.7 <= 90
constr(8):  X.1 + 5 X.2 + 3 X.3 + 10 X.4 + 2 X.5 + 4 X.6 + 9 X.7 <= 70
constr(9):  9 X.1 + 6 X.2 + X.3 + 4 X.4 + 7 X.5 + 5 X.6 + 10 X.7 <= 60
constr(10):  5 X.1 + 7 X.2 + 4 X.3 + 4 X.4 + 3 X.5 + 4 X.6 + 10 X.7 <= 40
Bounds
 0 <= X.1 <= 1
 0 <= X.2 <= 1
 0 <= X.3 <= 1
 0 <= X.4 <= 1
 0 <= X.5 <= 1
 0 <= X.6 <= 1
 0 <= X.7